In [1]:
import numpy as np
import pandas as pd
import os
import random
import cv2
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from skimage.transform import rotate

Using TensorFlow backend.


## Get files from a path

In [2]:
def get_files(path):
    files = list()
    
    # /kaggle/input/dataset/train
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    
    return files

In [3]:
train_files_path = get_files('/kaggle/input/dance-form-vaibhav/dataset/train/')

train_csv = pd.read_csv('/kaggle/input/dance-form-vaibhav/dataset/train.csv')

print("Train files : {0}".format(len(train_files_path)))

Train files : 364


In [4]:
train_dict = dict(zip(train_csv.Image, train_csv.target))
print(train_dict['96.jpg'])

manipuri


### Split into training and validation set

In [5]:
split = int(len(train_files_path) * 0.83)
random.shuffle(train_files_path)
train_files = train_files_path[:split]
val_files = train_files_path[split:]

print("Training files {0}".format(len(train_files)))
print("Validation files {0}".format(len(val_files)))

Training files 302
Validation files 62


### Converting dance forms into numbers

In [6]:
unique_labels = list(set(train_dict.values()))
unique_labels_mapping = dict()
for index in range(len(unique_labels)):
    unique_labels_mapping[unique_labels[index]] = index

total_classes = len(unique_labels)
print("Number of classes : {0}".format(total_classes))

Number of classes : 8


In [7]:
def get_one_hot_encoded_mask(value, num_labels):
    return to_categorical(value, num_classes = num_labels)

### Setting up hyper-parameters

In [8]:
img_width, img_height = 256, 256
batch_size = 16
epochs = 10
learning_rate = 1e-3

### Creating Augmentations

In [9]:
def rotate_image(image, angle_list):
    rotated_images = list()
    for angle in angle_list:
        rotated_images.append(rotate(image,angle))
    
    return rotated_images

In [10]:
def scaleDown_image(image, fx=0.6, fy=0.6):
    return cv2.resize(image, None, fx= 0.6, fy= 0.6, interpolation= cv2.INTER_LINEAR)

In [11]:
def scaleUp_image(image, fx = 2, fy = 2):
    return cv2.resize(image, None, fx = fx, fy = fy, interpolation= cv2.INTER_LINEAR)

In [12]:
def horizontal_flip(img):
    return img[: , ::-1]

## Creating DataGenerator

In [13]:
class DataGenerator:
    def __init__(self, train_files, valid_files, labels_dict, batch_size = 16, val_augment = True):
        self.train_files = train_files
        self.valid_files = valid_files
        self.labels_dict = labels_dict
        self.batch_size = batch_size
        self.val_augment = val_augment
        
    def train_generator(self):
        num_images = len(self.train_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                x_batch = list()
                y_batch = list()
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.train_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    img = img / 255
                    x_batch.append(img)
                    
                    image_name = self.train_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name[-1])]]
                    label = get_one_hot_encoded_mask(label, total_classes)
                    y_batch.append(label)
                    
                    rotated_images = rotate_image(img, [45, 60, -45, -60])
                    for rotated_image in rotated_images:
                        x_batch.append(rotated_image)
                        y_batch.append(label)
                                        
                    x_batch.append(horizontal_flip(img))
                    y_batch.append(label)
                
                x_batch, y_batch = shuffle(x_batch, y_batch, random_state=0)
                yield (np.asarray(x_batch), np.asarray(y_batch))
    
    def valid_generator(self):
        num_images = len(self.valid_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                x_batch = list()
                y_batch = list()
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.valid_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    img  = img / 255
                    x_batch.append(img)
                    
                    image_name = self.valid_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name[-1])]]
                    label = get_one_hot_encoded_mask(label, total_classes)
                    y_batch.append(label)
                    
                    if self.val_augment:
                        rotated_images = rotate_image(img, [45, 60, -45, -60])
                        for rotated_image in rotated_images:
                            x_batch.append(rotated_image)
                            y_batch.append(label)

                        x_batch.append(horizontal_flip(img))
                        y_batch.append(label)
                        
                x_batch, y_batch = shuffle(x_batch, y_batch, random_state=0)
                yield (np.asarray(x_batch), np.asarray(y_batch))

In [14]:
epoch_steps = int(math.ceil(len(train_files) / batch_size) * 6)
print(epoch_steps)
val_steps = int(math.ceil(len(val_files) / batch_size) * 6)
print(val_steps)

114
24


## Model Experiments

In [15]:
from keras.applications import vgg19, resnet50, inception_v3, inception_resnet_v2, mobilenet_v2
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.models import Model
import tensorflow.keras as keras

### VGG19 Setup

In [ ]:
model = vgg19.VGG19(include_top=False, input_shape=(img_width, img_height, 3), \
                    weights='/kaggle/input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.summary()

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

In [ ]:
model=Model(inputs=model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

### ResNet50 Setup

In [ ]:
base_model = resnet50.ResNet50(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=20, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

### InceptionV3 Setup

In [ ]:
base_model = inception_v3.InceptionV3(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu', kernel_regularizer='l1')(x)
x = Dropout(0.5)(x)
x = Dense(256,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
def lr_scheduler(epoch, lr):
#     if epoch % 2 == 0:
#         return lr
    return lr * 1.0

In [ ]:
callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
#     keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/models.{epoch:03d}.hdf5',
#                     monitor='loss', verbose=2, save_best_only=False)
]

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=20, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2, callbacks=callbacks)

### InceptionResnetV2 Setup

In [ ]:
base_model = inception_resnet_v2.InceptionResNetV2(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

### MobileNetV2 Setup

In [ ]:
base_model = mobilenet_v2.MobileNetV2(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/mobilenet-v2-128/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

### DeXpression Setup

In [16]:
def my_FeatCNN(input_shape,classes):
    padding = 'valid'
    img_input = keras.layers.Input(shape=input_shape)

    # START MODEL
    conv_1 = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding=padding, activation='relu', name='conv_1')(img_input)
    maxpool_1 = keras.layers.MaxPooling2D((2, 2), strides=(2,2))(conv_1)
    x = tf.keras.layers.BatchNormalization()(maxpool_1)
    
    # FEAT-EX1
    conv_2a = tf.keras.layers.Conv2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_2a')(x)
    conv_2b = tf.keras.layers.Conv2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_2b')(conv_2a)
    maxpool_2a = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_2a')(x)
    conv_2c = tf.keras.layers.Conv2D(64, (1, 1), strides=(1,1), name='conv_2c')(maxpool_2a)
    concat_1 = tf.keras.layers.concatenate(inputs=[conv_2b,conv_2c], axis=3,name='concat2')
    maxpool_2b = keras.layers.MaxPooling2D((3,3), strides=(2,2), padding=padding, name='maxpool_2b')(concat_1)

    # FEAT-EX2
    conv_3a = tf.keras.layers.Conv2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_3a')(maxpool_2b)
    conv_3b = tf.keras.layers.Conv2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_3b')(conv_3a)
    maxpool_3a = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3a')(maxpool_2b)
    conv_3c = tf.keras.layers.Conv2D(64, (1, 1), strides=(1,1), name='conv_3c')(maxpool_3a)
    concat_3 = tf.keras.layers.concatenate(inputs=[conv_3b,conv_3c],axis=3,name='concat3')
    maxpool_3b = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3b')(concat_3)
    
    # FEAT-EX3
#     conv_4a = tf.keras.layers.Conv2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_4a')(maxpool_3b)
#     conv_4b = tf.keras.layers.Conv2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_4b')(conv_4a)
#     maxpool_4a = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_4a')(maxpool_3b)
#     conv_4c = tf.keras.layers.Conv2D(64, (1, 1), strides=(1,1), name='conv_4c')(maxpool_4a)
#     concat_4 = tf.keras.layers.concatenate(inputs=[conv_4b,conv_4c],axis=3,name='concat4')
#     maxpool_4b = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_4b')(concat_4)
    
    # FINAL LAYERS
    net = tf.keras.layers.Flatten()(maxpool_3b)
    
    x = Dense(512,activation='relu')(net)
    x = Dense(256,activation='relu')(x)
    final_layer = Dense(classes,activation='softmax')(x)
    
    # Create model.
    model = tf.keras.Model(img_input, final_layer, name='deXpression')
    return model

In [17]:
model = my_FeatCNN((img_width,img_height,3), total_classes)
model.summary()

Model: "deXpression"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 126, 126, 64) 4864        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 63, 63, 64)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 63, 63, 64)   256         max_pooling2d[0][0]              
________________________________________________________________________________________

In [18]:
model.compile(optimizer=SGD(learning_rate=0.001 , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [20]:
def lr_scheduler(epoch, lr):
#     if epoch % 2 == 0:
#         return lr
    return lr * 1.0

In [21]:
callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
#     keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/models.{epoch:03d}.hdf5',
#                     monitor='loss', verbose=2, save_best_only=False)
]

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=20, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2, callbacks=callbacks)

Epoch 1/20
114/114 - 92s - loss: 1.8071 - accuracy: 0.3415 - val_loss: 2.0180 - val_accuracy: 0.2151 - lr: 0.0010
Epoch 2/20
114/114 - 94s - loss: 0.9423 - accuracy: 0.6720 - val_loss: 1.9147 - val_accuracy: 0.3244 - lr: 0.0010
Epoch 3/20
114/114 - 92s - loss: 0.3882 - accuracy: 0.8844 - val_loss: 1.4811 - val_accuracy: 0.5193 - lr: 0.0010
Epoch 4/20
114/114 - 93s - loss: 0.0949 - accuracy: 0.9846 - val_loss: 1.5305 - val_accuracy: 0.5475 - lr: 0.0010
Epoch 5/20


## Generate Predictions

In [ ]:
inv_map = {v: k for k, v in unique_labels_mapping.items()}
print(inv_map)

In [ ]:
test_csv = pd.read_csv('/kaggle/input/dance-form-vaibhav/dataset/test.csv')

In [ ]:
test_images = test_csv['Image']
print(test_images[:5])

In [ ]:
output = list()
for index in range(len(test_images)):
    img = cv2.imread('/kaggle/input/dance-form-vaibhav/dataset/test/' + str(test_images[index]))
    img = cv2.resize(img, (img_width, img_height))
    img = img / 255
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    output.append(inv_map[np.argmax(pred)])

In [ ]:
df = pd.DataFrame({'Image': test_images,
                   'target': output})
df.to_csv("/kaggle/working/output.csv", index=False)